<span type="title">工厂模式</span> | <span type="update">2018-09-02</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章介绍工厂模式</p></span>

# 工厂模式

## 需要解决的问题

一般而言，创建对象时使用 `new XXX()` 这种调用具体类来创建，但是这不满足 “面向接口而不是具体实现编程”，因此**将创建对象的代码单独封装起来**，将变化和不变隔离开来，这就是工厂模式。

工厂模式涉及两种情况，单独的创建一个具体对象，根据枚举或者字符串使用条件语句从一堆对象中选择创建一个具体对象。

**简要介绍**

所有的工厂都是用来封装对象的创建的。大致分为三种，简单工厂，工厂方法和抽象工厂。

对于简单工厂，其虽然不是设计模式，但是是一种将客户程序从具体类解耦的简单方法。工厂方法使用继承，把对象的实现委托给子类，子类实现工厂方法来创建对象。常见的工厂方法含有：`abstract Object createObject(String args)` 这种构造。抽象工厂使用对象的组合，对象的创建依然在子类的工厂中被实现（工厂方法）。其实可以将抽象工厂看作工厂方法的某种固定结构。

## 简单工厂

```java
public class PizzaStore {
    
    SimplePizzaFactory factory;
    PizzaStore(SimplePizzaFactory factory) { factory = factory; }
    Pizza orderPizza(String pizzaType) {
        //通过调用工厂方法来获得Pizza对象
        Pizza pizza = factory.createPizza(pizzaType);
        pizza.prepare();
        pizza.bake();
        pizza.cut();
        pizza.box();
        return pizza;
    }
}
```

**简单工厂的定义**

这种模式被称之为简单工厂（不是一种设计模式，更多的是设计习惯）。这种模式将客户端代码和具体创建对象的过程区分开来，实现了解耦。SimplePizzaFactory通过提供createPizza接口来被PizzaStore调用生产Pizza。

**简单工厂的实现**

简单工厂包括三个类，其一为工厂类，其二为流程类，其三为产品类。工厂通过`createObject`返回产品对象，在流程类中的`useObject`中调用产品的方法来处理对象，最后返回处理结果。

为了做到这个模式，势必要在流程类中组合保存工厂的实例，一般使用构造器来传入工厂对象。然后在具体的处理流程中调用工厂对象的实例返回产品对象。

**简单工厂的问题**

简单工厂的问题在于，创造产品的位置在两个地方，其一是工厂中，其二是流程中，这并不方便控制和统一。

因此，下面介绍的方法使用框架——真正的工厂方法来解决这个问题：使用继承而不是组合，让工厂类和流程类融为一体，在父类烘培，但是对象在子类才被定义和初始化。

下面提供一些辅助代码：

```java
public abstract class Pizza {
    String name;
    String dough;
    String sauce;
    ArrayList toppings = new ArrayList();
    void prepare() {
        System.out.println("Preparing " + name);
        System.out.println("Tossing dough... \n" +
        "Adding sauce... \n Adding toppings: \n");
        for (int i = 0; i < toppings.size(); i++) {
            System.out.println(" " + toppings.get(i));
        }
    }
    void bake() {System.out.println("Bake for 25 minutes ");}
    void cut() {System.out.println("Cutting now...");}
    void box() {System.out.println("Boxing now...");}
    public String getName() { return name; }
}

public class NYStyleCheesePizza extends Pizza {
    NYStyleCheesePizza() {
        name = "NYStyleCheesePizza";
        dough = "Extra Thick Crust Dough";
        sauce = "Plum Tomato Sauce";
        toppings.add("Shredded Mozzarella Cheese");
    }
    @Override void cut() {
        System.out.println("Cutting into square slices");
    }
}

public class NYStyleVeggiePizza extends Pizza {
    NYStyleVeggiePizza() {
        name = "NYStyleVeggiePizza";
        dough = "Extra Thick Crust Dough";
        sauce = "Plum Tomato Sauce";
        toppings.add("Shredded Mozzarella Veggie");
    }
    @Override void cut() {
        System.out.println("Cutting into three slices");
    }
}
```

## 工厂方法

**工厂方法的定义**

工厂方法指的是，**将对象的实现委托给子类，子类实现工厂方法来创建对象。**

这样完成了抽象工厂中需要复用的代码 `orderPizza` 和具体子类工厂中`createPizza` 实现的对象的分离，同时因为子类工厂继承了抽象工厂 `orderPizza` 的代码，因此在没有修改抽象工厂的基础上做到了抽象工厂代码的扩展。


```java
public abstract class PizzaStore {
    abstract Pizza createPizza(String pizzaType);
    Pizza orderPizza(String pizzaType) {
        Pizza pizza = createPizza(pizzaType);
        pizza.prepare();
        pizza.bake();
        pizza.cut();
        pizza.box();
        return pizza;
    }
}

public class NYPizzaStore extends PizzaStore {
    Pizza createPizza(String pizzaType) {
        if (pizzaType.equals("cheese")) {
            return new NYStyleCheesePizza();
        } else if (pizzaType.equals("veggie")) {
            return new NYStyleVeggiePizza();
        } else return null;
    }
}
//Run here:
PizzaStore nyStore = new NYPizzaStore();
Pizza pizza = nyStore.orderPizza("cheese");
Pizza pizza = nyStore.orderPizza("veggie");
```

## 依赖倒置原则

依赖倒置原则指的是：**要依赖抽象，而不是依赖具体实现**。

这里的规则和面向接口编程而不是面向具体实例几乎含义相同，但是侧重点不同，这里更强调不要依赖具体的类。而之前那句话的含义主要是，要面向接口编程。

对于本例子而言，依赖抽象而不是具体类的含义就是：不要在商店里实例化各种 Pizza 类型，而应该将其交给工厂去做。

对于简单工厂，工厂和商店是两个类，后者保存前者实例进行组合，我们现在面向的是一个统一的门店。

对于工厂方法，工厂即是商店。工厂方法的定义是：通过委托子类实现对象初始化的方法，来保证了自己的代码不被修改的基础上能够扩展产生不同对象。在这种情况下，依赖抽象而不是具体类指的就是内部的`abstract createObject`和`orderObject`分离。这样的话，抽象的PizzaStore就可以完全依赖 Pizza 而不用担心依赖具体的Pizza类型的问题了（因为实现被通过工厂方法委托给了子类，所以抽象的PizzaStore依赖倒置）。

对于本例而言，PizzaStore仅依赖Pizza抽象，各种不同的Pizza也只依赖Pizza抽象。这就是依赖倒置原则（不论是高级别还是低级别组建都依赖相同的抽象）。

**使用依赖倒置原则进行思考**

依赖倒置原则的思考方式：创建一个商店，商店售卖不同的商品。因为**不能依赖具体对象**，因此我们抽象出一个商品类，然后让这些商品继承自商品抽象类。这样商店就不依赖具体类了。为了区别这些抽象类，因此还需要使用工厂。

简单工厂清晰易懂，但是没有很好的整合变化和不变的部分，因此，使用工厂方法，在子类中实现从抽象商品类中获得具体商品对象的操作。这样商店的代码和商店子类初始化具体对象的代码分离，就做到了**商店向下依赖抽象商品类，各种具体商品类向上依赖抽象商品类**, 这就是依赖倒置。

**依赖倒置原则的良好实践**

注意，我们不能完全做到这条，因为在商店子类不可避免的要通过new方法来创建具体商品对象，这是Java语法决定的。但是，要尽量的做到：

- 变量不持有具体类的引用，而是要持有抽象接口的
- 类不要派生自具体的类，而是要派生自抽象类或者使用接口
- 不要覆盖基类中已经实现的方法

这三条可以保证依赖倒置，完成干净的OOP设计。但是实际其实很难做到。

现在我们还有一个需求：如果商店不仅仅生产Pizza呢，这很简单，使用 `abstract Object createObject(String...args)` 创建一个新的工厂方法即可。

这就是抽象工厂设计模式。

## 抽象工厂

**抽象工厂的定义**

抽象工厂指的是**提供一个接口，用于创建相关或者依赖对象的整个家族，而不需要指明具体的类**。

**区别抽象工厂和工厂方法**

抽象工厂定义了一个包含众多工厂方法的接口结构。似乎抽象工厂包含了工厂方法。但是，我们在这里澄清一个点：这里的工厂方法指的不仅仅是那个抽象的工厂方法，还指的是将产生对象和使用方法放在同一个类-子类结构中的做法。

相比较只包含一个工厂方法的 PizzaStore ，抽象工厂模式将混合获得和使用对象从 PizzaStore 及其子类中拆分开来。虽然之前的方法——在抽象的PizzaSotre类中定义使用方法，而在子类中实现工作方法接口可以保证无修改性和扩展性。

这样看来，我们似乎在重复从简单工厂到工厂方法的逆过程，而这样做的最大弊端和使用简单工厂一样，就是在多处进行了更改，createObject 和 orderObject，将其放在一处更容易修改。因此，何时使用he'zhe

因此，对于只有一个或者固定较少个数的工厂方法，使用纯粹的工厂方法，使用继承和abstract产生对象的方法来保证无修改性和扩展性，而不是试图将产生对象和使用对象进行拆分更好。而对于需要产生固定个数的多个工厂方法的例子，将产生对象和使用对象拆分开会更加清晰。比如，这个Pizza的例子的一个问题就是：我们现在为每个地域都制作了很多Pizza，比如NY州的芝士披萨和素菜披萨，如果有别的州，它们的制作可以重用这些披萨类，但是现在，我们想要一点不同风味的，但是标准相同的披萨，就像麦当劳，各地的麦当劳生产同样名称的东西，不过原产地不同，这时就需要使用抽象工厂方法了。

```java
abstract class Dough {}
class ThinCrustDough extends Dough {}
abstract class Sauce {}
class MarinaraSauce extends Sauce {}
abstract class Cheese {}
class ChineseCheese extends Cheese {}

public interface PizzaIngredientFactory {
    public Dough createDough();
    public Sauce createSauce();
    public Cheese creteaCheese();
}

public class NYPizzaIngredientFactory implements PizzaIngredientFactory {
    public Dough createDough() {
        return new ThinCrustDough();
    }
    public Sauce createSauce() {
        return new MarinaraSauce();
    }
    public Cheese createCheese() {
        return new ChineseCheese();
    }
}
```